In [14]:
import numpy as np
import random

In [59]:
class Board():
    
    def __init__(self, deck):
        
        self.deck = random.sample(deck, 5)
        self.player1_cards = self.deck[:2]
        self.player2_cards = self.deck[2:4]
        self.remaining_card = self.deck[4]
        self.board_size = 5 # 5x5 board 
        self.board_state = np.array([[ 1,  1,  2,  1,  1],
                                     [ 0,  0,  0,  0,  0],
                                     [ 0,  0,  0,  0,  0],
                                     [ 0,  0,  0,  0,  0],
                                     [-1, -1, -2, -1, -1]])
        
        
    def get_legal_moves(self, player):
        """Returns all the legal moves for the given player """
        moves = set()  # stores the legal moves.

        if player == 1:
            pass
        
        else : 
            pass

    
    def is_win(self, player):
        """Check whether the given player has win"""
        
        # Player1 win if -2 is dead, or if 2 managed to reach the center of the last line
        if player == 1:
            
            if self.board_state[4][2] == 2:
                return True
            
            for i in range(self.board_size):
                for j in range(self.board_size):
                    if self.board_state[i][j] == -2:
                        return False
            return True
        
        # Player2 (-1) win if 2 is dead, or if -2 managed to reach the center of the first line
        else:
            if self.board_state[0][2] == -2:
                return True
            
            for i in range(self.board_size):
                for j in range(self.board_size):
                    if self.board_state[i][j] == 2:
                        return False
            return True
        

    def execute_move(self, move, color):
        """Perform the given move on the board"""

        pass